En este script procesamos el dataset clickbait_dataset_multitoken para extraer los titulares de las noticias gracias a la urls extraídas en el script ObtencionTweetsClickbait.



In [ ]:
import pandas as pd
import requests
import time
import re
from datetime import datetime, timedelta
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

class MultiTokenTwitterProcessor:
    def __init__(self, bearer_tokens):
        self.bearer_tokens = bearer_tokens
        self.token_count = len(bearer_tokens)
        self.headers_list = []

        # Crear headers para cada token
        for token in bearer_tokens:
            self.headers_list.append({
                'Authorization': f'Bearer {token}',
                'Content-Type': 'application/json'
            })

        self.processed_count = 0
        self.start_time = datetime.now()
        self.session_stats = {
            'successful': 0,
            'errors': 0,
            'rate_limited': 0,
            'not_found': 0
        }

        # Lock para thread safety
        self.stats_lock = threading.Lock()

    def extract_tweet_id(self, url):
        """
        Extrae el ID del tweet de una URL de Twitter
        """
        if pd.isna(url) or url == '':
            return None

        patterns = [
            r'twitter\.com/[^/]+/status/(\d+)',
            r'x\.com/[^/]+/status/(\d+)',
            r'mobile\.twitter\.com/[^/]+/status/(\d+)'
        ]

        for pattern in patterns:
            match = re.search(pattern, str(url))
            if match:
                return match.group(1)
        return None

    def get_tweet_text_with_token(self, tweet_id, token_index):
        """
        Obtiene el texto de un tweet usando un token específico
        """
        url = f"https://api.twitter.com/2/tweets/{tweet_id}"
        params = {
            'tweet.fields': 'text,created_at,author_id,public_metrics'
        }

        headers = self.headers_list[token_index]

        try:
            response = requests.get(url, headers=headers, params=params, timeout=30)

            if response.status_code == 200:
                data = response.json()
                if 'data' in data:
                    with self.stats_lock:
                        self.session_stats['successful'] += 1
                    return {
                        'text': data['data']['text'],
                        'status': 'success',
                        'token_used': token_index + 1
                    }
                else:
                    with self.stats_lock:
                        self.session_stats['errors'] += 1
                    return {
                        'text': "Error: No se encontró el tweet",
                        'status': 'no_data',
                        'token_used': token_index + 1
                    }
            elif response.status_code == 429:
                with self.stats_lock:
                    self.session_stats['rate_limited'] += 1
                return {
                    'text': "Error: Límite de rate excedido",
                    'status': 'rate_limited',
                    'token_used': token_index + 1
                }
            elif response.status_code == 404:
                with self.stats_lock:
                    self.session_stats['not_found'] += 1
                return {
                    'text': "Error: Tweet no encontrado o privado",
                    'status': 'not_found',
                    'token_used': token_index + 1
                }
            else:
                with self.stats_lock:
                    self.session_stats['errors'] += 1
                return {
                    'text': f"Error HTTP: {response.status_code}",
                    'status': 'http_error',
                    'token_used': token_index + 1
                }

        except requests.exceptions.Timeout:
            with self.stats_lock:
                self.session_stats['errors'] += 1
            return {
                'text': "Error: Timeout de conexión",
                'status': 'timeout',
                'token_used': token_index + 1
            }
        except Exception as e:
            with self.stats_lock:
                self.session_stats['errors'] += 1
            return {
                'text': f"Error de conexión: {str(e)}",
                'status': 'connection_error',
                'token_used': token_index + 1
            }

    def process_tweet_batch(self, tweet_data_batch):
        """
        Procesa un lote de tweets usando diferentes tokens simultáneamente
        """
        results = {}

        # Usar ThreadPoolExecutor para procesar en paralelo
        with ThreadPoolExecutor(max_workers=self.token_count) as executor:
            # Crear tareas para cada tweet
            future_to_data = {}
            for i, (idx, url) in enumerate(tweet_data_batch):
                tweet_id = self.extract_tweet_id(url)
                if tweet_id:
                    token_index = i % self.token_count  # Distribuir tokens
                    future = executor.submit(self.get_tweet_text_with_token, tweet_id, token_index)
                    future_to_data[future] = (idx, url, tweet_id, token_index)
                else:
                    results[idx] = {
                        'text': "Error: No se pudo extraer ID del tweet",
                        'status': 'invalid_url',
                        'token_used': 'N/A'
                    }

            # Recoger resultados
            for future in as_completed(future_to_data):
                idx, url, tweet_id, token_index = future_to_data[future]
                try:
                    result = future.result()
                    results[idx] = result
                except Exception as e:
                    results[idx] = {
                        'text': f"Error en thread: {str(e)}",
                        'status': 'thread_error',
                        'token_used': token_index + 1
                    }

        return results

    def save_progress(self, df, batch_num=None, is_final=False):
        """
        Guarda el progreso del procesamiento
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        if is_final:
            filename = f'dataset_procesado_FINAL_{timestamp}.csv'
        elif batch_num:
            filename = f'dataset_procesado_lote_{batch_num}_{timestamp}.csv'
        else:
            filename = f'dataset_procesado_progreso_{timestamp}.csv'

        df.to_csv(filename, index=False)
        print(f"💾 Progreso guardado en: {filename}")
        return filename

    def load_existing_progress(self, original_df):
        """
        Carga progreso existente si hay archivos de guardado previos
        """
        import glob
        import os

        # Buscar archivos de progreso existentes
        progress_files = glob.glob('dataset_procesado_*.csv')

        if not progress_files:
            print("📂 No se encontraron archivos de progreso previos")
            return original_df

        # Encontrar el archivo más reciente
        latest_file = max(progress_files, key=os.path.getctime)

        try:
            print(f"📂 Cargando progreso desde: {latest_file}")
            df_progress = pd.read_csv(latest_file)

            # Verificar que tiene las columnas necesarias
            required_cols = ['texto_titular', 'token_usado', 'status_procesamiento']
            if all(col in df_progress.columns for col in required_cols):
                # Contar tweets procesados
                processed_count = df_progress[
                    (df_progress['texto_titular'].notna()) &
                    (df_progress['texto_titular'] != '') &
                    (df_progress['texto_titular'] != 'Error: No se pudo extraer ID del tweet')
                ].shape[0]

                print(f"✅ Progress cargado exitosamente!")
                print(f"   Tweets ya procesados: {processed_count}")
                return df_progress
            else:
                print("⚠️  Archivo de progreso no tiene las columnas necesarias, iniciando desde cero")
                return original_df

        except Exception as e:
            print(f"❌ Error al cargar progreso: {str(e)}")
            print("   Iniciando desde cero...")
            return original_df

    def process_dataset_multi_token(self, df, delay_minutes=15, batch_size=None, max_batches=None, save_every=5):
        """
        Procesa el dataset usando múltiples tokens con guardado automático
        """
        if batch_size is None:
            batch_size = self.token_count  # Por defecto, usar tantos tweets como tokens

        # Intentar cargar progreso existente
        df = self.load_existing_progress(df)

        # Crear las columnas si no existen
        if 'texto_titular' not in df.columns:
            df['texto_titular'] = ''
        if 'token_usado' not in df.columns:
            df['token_usado'] = ''
        if 'status_procesamiento' not in df.columns:
            df['status_procesamiento'] = ''

        # Encontrar tweets no procesados
        mask = (df['texto_titular'].isna()) | (df['texto_titular'] == '') | (df['texto_titular'] == 'Error: No se pudo extraer ID del tweet')
        pending_indices = df[mask].index.tolist()

        if not pending_indices:
            print("🎉 ¡Todos los tweets ya han sido procesados!")
            final_file = self.save_progress(df, is_final=True)
            return df

        # Contar tweets ya procesados
        total_tweets = len(df)
        processed_tweets = total_tweets - len(pending_indices)

        print(f"📊 ESTADÍSTICAS INICIALES:")
        print(f"   Total de URLs: {total_tweets}")
        print(f"   URLs ya procesadas: {processed_tweets}")
        print(f"   URLs pendientes: {len(pending_indices)}")
        print(f"   Progreso actual: {(processed_tweets/total_tweets*100):.1f}%")
        print(f"   Tokens disponibles: {self.token_count}")
        print(f"   Tweets por lote: {batch_size}")
        print(f"   Delay entre lotes: {delay_minutes} minutos")
        print(f"   Guardado automático cada: {save_every} lotes")
        print("-" * 60)

        # Crear lotes
        batches = []
        for i in range(0, len(pending_indices), batch_size):
            batch_indices = pending_indices[i:i+batch_size]
            batch_data = [(idx, df.loc[idx, 'titular_noticia']) for idx in batch_indices]
            batches.append(batch_data)

        if max_batches:
            batches = batches[:max_batches]

        print(f"📦 Se procesarán {len(batches)} lotes")
        if max_batches:
            print(f"   (Limitado a {max_batches} lotes por configuración)")

        total_estimated_time = (len(batches) - 1) * delay_minutes
        estimated_finish = datetime.now() + timedelta(minutes=total_estimated_time)
        print(f"⏱️  Tiempo estimado: {total_estimated_time} minutos")
        print(f"   Finalización estimada: {estimated_finish.strftime('%H:%M:%S')}")
        print("=" * 60)

        # Procesar lotes
        for batch_num, batch_data in enumerate(batches, 1):
            print(f"\n🚀 LOTE {batch_num}/{len(batches)} - {datetime.now().strftime('%H:%M:%S')}")
            print(f"   Procesando {len(batch_data)} tweets...")

            # Mostrar URLs del lote
            for i, (idx, url) in enumerate(batch_data):
                print(f"   [{i+1}] Fila {idx}: {url}")

            batch_start = time.time()
            results = self.process_tweet_batch(batch_data)
            batch_duration = time.time() - batch_start

            # Actualizar DataFrame con resultados
            for idx, result in results.items():
                df.loc[idx, 'texto_titular'] = result['text']
                df.loc[idx, 'token_usado'] = str(result['token_used'])
                df.loc[idx, 'status_procesamiento'] = result['status']

            # Mostrar resultados del lote
            print(f"\n📊 RESULTADOS DEL LOTE {batch_num}:")
            success_count = sum(1 for r in results.values() if r['status'] == 'success')
            print(f"   ✅ Exitosos: {success_count}/{len(batch_data)}")
            print(f"   ⏱️  Tiempo del lote: {batch_duration:.1f} segundos")

            # Mostrar algunos ejemplos de texto obtenido
            successful_results = [(idx, r) for idx, r in results.items() if r['status'] == 'success']
            if successful_results:
                print(f"   📝 Ejemplos de texto obtenido:")
                for idx, result in successful_results[:2]:  # Mostrar máximo 2 ejemplos
                    text_preview = result['text'][:80] + "..." if len(result['text']) > 80 else result['text']
                    print(f"      Fila {idx}: {text_preview}")

            # Guardar progreso cada X lotes
            if batch_num % save_every == 0:
                print(f"\n💾 Guardando progreso automático (lote {batch_num})...")
                progress_file = self.save_progress(df, batch_num)

                # Mostrar estadísticas de progreso
                total_processed_now = len(df) - len(df[(df['texto_titular'].isna()) |
                                                      (df['texto_titular'] == '') |
                                                      (df['texto_titular'] == 'Error: No se pudo extraer ID del tweet')])
                progress_pct = (total_processed_now / len(df)) * 100
                print(f"   📈 Progreso total: {total_processed_now}/{len(df)} ({progress_pct:.1f}%)")
                print(f"   📁 Archivo: {progress_file}")

            # Esperar antes del siguiente lote (excepto en el último)
            if batch_num < len(batches):
                print(f"\n⏸️  Esperando {delay_minutes} minutos antes del siguiente lote...")
                print(f"   Siguiente lote a las: {(datetime.now() + timedelta(minutes=delay_minutes)).strftime('%H:%M:%S')}")

                # Guardado de seguridad cada 30 minutos (2 lotes)
                if batch_num % 2 == 0:
                    backup_file = self.save_progress(df, batch_num, is_final=False)
                    print(f"   🛡️  Backup de seguridad: {backup_file}")

                time.sleep(delay_minutes * 60)

        # Estadísticas finales
        elapsed_time = datetime.now() - self.start_time
        total_processed = sum(self.session_stats.values())

        print(f"\n" + "=" * 60)
        print(f"🎉 PROCESAMIENTO COMPLETADO!")
        print(f"⏱️  Tiempo total: {elapsed_time}")
        print(f"📊 ESTADÍSTICAS FINALES:")
        print(f"   ✅ Exitosos: {self.session_stats['successful']}")
        print(f"   ❌ Errores: {self.session_stats['errors']}")
        print(f"   🚫 Rate limited: {self.session_stats['rate_limited']}")
        print(f"   🔍 No encontrados: {self.session_stats['not_found']}")
        print(f"   📊 Total procesados en esta sesión: {total_processed}")
        print(f"   🎯 Tasa de éxito: {(self.session_stats['successful']/total_processed*100):.1f}%" if total_processed > 0 else "   🎯 Tasa de éxito: 0%")

        # Guardar archivo final
        final_file = self.save_progress(df, is_final=True)
        print(f"💾 Archivo final guardado: {final_file}")

        # Crear resumen de estados
        try:
            summary_df = df['status_procesamiento'].value_counts().to_frame()
            summary_filename = f'resumen_procesamiento_{datetime.now().strftime("%Y%m%d_%H%M")}.csv'
            summary_df.to_csv(summary_filename)
            print(f"📊 Resumen de estados: {summary_filename}")
        except Exception as e:
            print(f"⚠️  No se pudo generar resumen: {str(e)}")

        print("=" * 60)

        return df

# CONFIGURACIÓN PRINCIPAL
BEARER_TOKENS = [
    "AAAAAAAAAAAAAAAAAAAAAO6%2BzQEAAAAAfryhIj%2BIG9KIbx3tMDhj6B0Tbm8%3DKVoE5N6ibJE5Zh8VuuCtGLkTnFe49EQRgjK2QwkuXmIAm7X4MB",
    "AAAAAAAAAAAAAAAAAAAAAFd62AEAAAAAYOBUuWTu18IxkOTfrOH50r%2BCqfc%3DbE7hjNoebVw9AvKpYDSr2AA4ooNaQO9Vua51nTttslJjndLCQv",
    "AAAAAAAAAAAAAAAAAAAAALN62AEAAAAArziUSqloB0%2BPSmM5tUuyp4HU%2BGU%3DL0m0vYE2iBgqABL5wUH1KcBlJbBOnhmWEPSYlydgY8eFJOBrCr",
    "AAAAAAAAAAAAAAAAAAAAAOF62AEAAAAA5aF%2F%2BM%2FgwXyCEDjYjtiCLV8ODqA%3DhicC7ZhR5SDJ6fl4vs6ztRmlMhWLDmftmv2vzkpE1HkUMKu1R2",
    "AAAAAAAAAAAAAAAAAAAAAOZ62AEAAAAAddltHTioEAGrHkvZlMuNwvuv4tg%3DHl6OodDftl5lFhC9vlykmg4f1H93TkeQ7L5yuN6evXmb94EPXP"
]

print("🔧 Inicializando procesador multi-token...")
print(f"   Tokens configurados: {len(BEARER_TOKENS)}")

# Cargar el dataset
print("📂 Cargando dataset...")
df = pd.read_csv('clickbait_dataset_multitoken_20250530_155914.csv')

print(f"📊 Dataset cargado: {len(df)} filas")
if 'titular_noticia' in df.columns:
    valid_urls = df['titular_noticia'].notna().sum()
    print(f"   URLs válidas: {valid_urls}")
else:
    print("❌ Error: Columna 'titular_noticia' no encontrada")
    exit()

# Crear el procesador
processor = MultiTokenTwitterProcessor(BEARER_TOKENS)

print("\n" + "="*60)
print("🚀 PROCESAMIENTO COMPLETO - MÚLTIPLES TOKENS")
print("="*60)

# Calcular cuántos lotes necesitamos para 454 URLs
total_urls = 454
batch_size = 5  # 5 tweets por lote (usando los 5 tokens)
estimated_batches = (total_urls + batch_size - 1) // batch_size  # Redondeo hacia arriba

print(f"📈 Para procesar {total_urls} URLs:")
print(f"   Lotes estimados: {estimated_batches}")
print(f"   Tiempo estimado: {(estimated_batches-1) * 15} minutos ({((estimated_batches-1) * 15) / 60:.1f} horas)")

# Procesar todo el dataset con guardado automático
df_final = processor.process_dataset_multi_token(
    df,
    delay_minutes=15,    # 15 minutos entre lotes
    batch_size=5,        # 5 tweets por lote
    max_batches=None,    # Sin límite, procesar todo
    save_every=5         # Guardar cada 5 lotes (cada 75 minutos)
)

print(f"\n🎉 ¡PROCESAMIENTO TERMINADO!")
print(f"📁 Revisa los archivos generados con 'dataset_procesado_' en el nombre")
print(f"🔄 Si el proceso se interrumpió, solo ejecuta el código de nuevo")
print(f"   y automáticamente continuará desde donde se quedó")

🔧 Inicializando procesador multi-token...
   Tokens configurados: 5
📂 Cargando dataset...
📊 Dataset cargado: 454 filas
   URLs válidas: 454

🚀 PROCESAMIENTO COMPLETO - MÚLTIPLES TOKENS
📈 Para procesar 454 URLs:
   Lotes estimados: 91
   Tiempo estimado: 1350 minutos (22.5 horas)
📂 Cargando progreso desde: dataset_procesado_lote_34_20250604_210402.csv
✅ Progress cargado exitosamente!
   Tweets ya procesados: 400
📊 ESTADÍSTICAS INICIALES:
   Total de URLs: 454
   URLs ya procesadas: 400
   URLs pendientes: 54
   Progreso actual: 88.1%
   Tokens disponibles: 5
   Tweets por lote: 5
   Delay entre lotes: 15 minutos
   Guardado automático cada: 5 lotes
------------------------------------------------------------
📦 Se procesarán 11 lotes
⏱️  Tiempo estimado: 150 minutos
   Finalización estimada: 23:43:30

🚀 LOTE 1/11 - 21:13:30
   Procesando 5 tweets...
   [1] Fila 400: https://twitter.com/libertaddigital/status/1911428141844677071
   [2] Fila 401: https://twitter.com/AristeguiOnline/status/

<ipython-input-1-a60943cbf9a6>:304: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, 'token_usado'] = str(result['token_used'])



📊 RESULTADOS DEL LOTE 1:
   ✅ Exitosos: 4/5
   ⏱️  Tiempo del lote: 0.2 segundos
   📝 Ejemplos de texto obtenido:
      Fila 401: #Video▶️ | La Ciudad de México confirma quién dará concierto gratuito en el Zóca...
      Fila 403: No será ni Miranda! ni Rod Stewart: esta es la superestrella mundial que inaugur...

⏸️  Esperando 15 minutos antes del siguiente lote...
   Siguiente lote a las: 21:28:31

🚀 LOTE 2/11 - 21:28:31
   Procesando 5 tweets...
   [1] Fila 405: https://twitter.com/sextaNoticias/status/1910935919986700779
   [2] Fila 406: https://twitter.com/okdiario/status/1910936995896971553
   [3] Fila 407: https://twitter.com/ahorrandoclick1/status/1910704116843569578/photo/1
   [4] Fila 408: https://twitter.com/soy_502/status/1910703539489268162
   [5] Fila 409: https://twitter.com/larazon_es/status/1910580321629675731

📊 RESULTADOS DEL LOTE 2:
   ✅ Exitosos: 2/5
   ⏱️  Tiempo del lote: 0.1 segundos
   📝 Ejemplos de texto obtenido:
      Fila 409: 🚔 La Guardia Civil comenzará a